In [1]:
import sys
sys.path.insert(1, '../')
from FrenetFDA.processing_Euclidean_curve.preprocessing import compute_arc_length, compute_derivatives
from FrenetFDA.processing_Euclidean_curve.estimate_Frenet_curvatures import ExtrinsicFormulas
from FrenetFDA.processing_Euclidean_curve.estimate_Frenet_path import GramSchmidtOrthogonalization, ConstrainedLocalPolynomialRegression
import FrenetFDA.utils.visualization as visu
from FrenetFDA.utils.Frenet_Serret_utils import solve_FrenetSerret_ODE_SE
import numpy as np
from scipy.integrate import cumtrapz, solve_ivp

INFO: Using numpy backend


In [2]:
def generate_Y(X, gamma):
    W = gamma**2*np.eye(3)
    Y = X + np.random.multivariate_normal(np.zeros(3), W, size=(len(X)))
    return Y

N = 200
grid = np.linspace(0,1,N)


""" __________ Definition of exact function Theta ___________ """

curv = lambda s : 2*np.cos(2*np.pi*s) + 5
tors = lambda s : 2*np.sin(2*np.pi*s) + 1

def theta(s):
    if isinstance(s, int) or isinstance(s, float):
        return np.array([curv(s), tors(s)])
    elif isinstance(s, np.ndarray):
        return np.vstack((curv(s), tors(s))).T
    else:
        raise ValueError('Variable is not a float, a int or a NumPy array.')
    
def warping(s,a):
    if np.abs(a)<1e-15:
        return s
    else:
        return (np.exp(a*s) - 1)/(np.exp(a) - 1)
    
warp_grid = warping(grid, 2)
    
""" __________ Definition of states and noisy observations __________ """

mu0 = np.eye(4)
Z = solve_FrenetSerret_ODE_SE(theta, warp_grid, Z0=mu0)
Q = Z[:,:3,:3]
X = Z[:,:3,3]

gamma = 0.001
W = gamma**2*np.eye(3)
Y = generate_Y(X, gamma)

In [3]:
visu.plot_3D([X, Y], ['X', 'Y'])

###
###
### Test preprocessing X

In [6]:
h_grid = np.array([0.005, 0.01, 0.015, 0.02, 0.1, 0.15, 1])
grid_arc_s, L, arc_s, arc_s_dot = compute_arc_length(Y, grid, scale=True, smoothing_param=None, CV_optimization={"flag":True, "h_grid":h_grid, "K":20})
visu.plot_array_2D_names(grid, [warp_grid, grid_arc_s, arc_s(grid), arc_s_dot(grid)], ['true_s', 'estim_s', 'estim_s_interp', 'estim_sdot'])

Optimal smoothing parameter h find by cross-validation: 0.15


In [8]:
time_derivatives = compute_derivatives(Y, grid, deg=4, h=0.1)
arc_length_derivatives_1 = compute_derivatives(Y, warp_grid, deg=4, h=0.1)
arc_length_derivatives_2 = compute_derivatives(Y, grid_arc_s, deg=4, h=0.1)
visu.plot_3D([Y, time_derivatives[0], X, arc_length_derivatives_1[0], arc_length_derivatives_2[0]], ["Y", "time_deriv_Y", "X", "arc_s_deriv_Y", "arc_s_deriv_Y_bis"])

In [10]:
xdot_t_bis = np.zeros((200,3))
sdot = arc_s_dot(grid)
for i in range(200):
    xdot_t_bis[i] = sdot[i]*arc_length_derivatives_2[1][i]
visu.plot_3D([time_derivatives[1], arc_length_derivatives_2[1], xdot_t_bis], ["xdot_t", "Xprime_s", "xdot_t_bis"], mode="markers")

###
###
### Test estimation theta extrinsic formulas

In [12]:
theta_extrins = ExtrinsicFormulas(Y, grid, warp_grid).raw_estimates(0.05)
visu.plot_array_2D_names(warp_grid, [theta_extrins[:,0], curv(warp_grid)], ['raw curv', 'true curv'])
visu.plot_array_2D_names(warp_grid, [theta_extrins[:,1], tors(warp_grid)], ['raw tors', 'true tors'])

In [17]:
Bspline_repres = ExtrinsicFormulas(Y, grid, warp_grid).Bspline_smooth_estimates(0.2, 30, regularization_parameter=0.1)
Bspline_repres.plot()

In [4]:
h_list = np.array([0.05, 0.1, 0.2, 0.3, 0.4])
nb_basis_list = np.array([20, 30])
regularization_parameter_list = np.array([0.1, 0.01, 0.001])
h_opt, nb_basis_opt, regularization_parameter_opt, tab_GCV_scores_1, error_bandwidth_1 = ExtrinsicFormulas(Y, grid, warp_grid).grid_search_optimization_hyperparameters(h_list, nb_basis_list, regularization_parameter_list, method='1')

Begin grid search optimisation with 30 combinations of parameters...
Optimal parameters selected by grid search optimisation:  bandwidth = 0.2 nb_basis = 30 regularization_parameter = 0.001


In [5]:
print(tab_GCV_scores_1)
print(error_bandwidth_1)

[[[1.46197727e+04 1.47380173e+04 1.50912536e+04]
  [4.99884751e+02 4.83521931e+02 4.60063936e+02]
  [5.77895063e+01 5.20784653e+01 4.28253525e+01]
  [1.47247955e+01 1.20647826e+01 1.07393769e+01]
  [4.90869841e+01 3.56357045e+01 2.14773495e+01]]

 [[1.46194418e+04 1.47351766e+04 1.50689080e+04]
  [4.99886458e+02 4.83500787e+02 4.59416962e+02]
  [5.77892332e+01 5.20644480e+01 4.25264518e+01]
  [1.47228695e+01 1.20525846e+01 1.06628379e+01]
  [4.90870893e+01 3.56329642e+01 2.13453712e+01]]]
[1.38173011 0.02992186 0.00724379 0.01103961 0.01255436]


In [3]:
h_list = np.array([0.05, 0.1, 0.2, 0.3, 0.4])
nb_basis_list = np.array([20, 30])
regularization_parameter_list = np.array([0.1, 0.01, 0.001])
h_opt, nb_basis_opt, regularization_parameter_opt, CV_error_tab = ExtrinsicFormulas(Y, grid, warp_grid).grid_search_optimization_hyperparameters(h_list, nb_basis_list, regularization_parameter_list, method='2')

Begin grid search optimisation with 30 combinations of parameters...
nb_basis: 20 h: 0.05 lbda: 0.1
nb_basis: 20 h: 0.05 lbda: 0.01
nb_basis: 20 h: 0.05 lbda: 0.001
nb_basis: 20 h: 0.1 lbda: 0.1
nb_basis: 20 h: 0.1 lbda: 0.01
nb_basis: 20 h: 0.1 lbda: 0.001
nb_basis: 20 h: 0.2 lbda: 0.1
nb_basis: 20 h: 0.2 lbda: 0.01
nb_basis: 20 h: 0.2 lbda: 0.001
nb_basis: 20 h: 0.3 lbda: 0.1
nb_basis: 20 h: 0.3 lbda: 0.01
nb_basis: 20 h: 0.3 lbda: 0.001
nb_basis: 20 h: 0.4 lbda: 0.1
nb_basis: 20 h: 0.4 lbda: 0.01
nb_basis: 20 h: 0.4 lbda: 0.001
nb_basis: 30 h: 0.05 lbda: 0.1
nb_basis: 30 h: 0.05 lbda: 0.01
nb_basis: 30 h: 0.05 lbda: 0.001
nb_basis: 30 h: 0.1 lbda: 0.1
nb_basis: 30 h: 0.1 lbda: 0.01
nb_basis: 30 h: 0.1 lbda: 0.001
nb_basis: 30 h: 0.2 lbda: 0.1
nb_basis: 30 h: 0.2 lbda: 0.01
nb_basis: 30 h: 0.2 lbda: 0.001
nb_basis: 30 h: 0.3 lbda: 0.1
nb_basis: 30 h: 0.3 lbda: 0.01
nb_basis: 30 h: 0.3 lbda: 0.001
nb_basis: 30 h: 0.4 lbda: 0.1
nb_basis: 30 h: 0.4 lbda: 0.01
nb_basis: 30 h: 0.4 lbda: 0

In [4]:
Bspline_repres = ExtrinsicFormulas(Y, grid, warp_grid).Bspline_smooth_estimates(0.2, 30, regularization_parameter=0.001)
Bspline_repres.plot()

In [5]:
Bspline_repres_2 = ExtrinsicFormulas(Y, grid, warp_grid).Bspline_smooth_estimates(0.2, 20, regularization_parameter=0.001)
Bspline_repres_2.plot()

In [6]:
visu.plot_array_2D(grid, [curv(grid), Bspline_repres.evaluate(grid)[:,0], Bspline_repres_2.evaluate(grid)[:,0]], '')
visu.plot_array_2D(grid, [tors(grid), Bspline_repres.evaluate(grid)[:,1], Bspline_repres_2.evaluate(grid)[:,1]], '')

###
###
### Test estimation Q 

In [3]:
Z_GS, Q_GS, X_GS = GramSchmidtOrthogonalization(Y, warp_grid).fit(h=0.1, reparam_grid=None)
Z_GS_reparam, Q_GS_reparam, X_GS_reparam = GramSchmidtOrthogonalization(Y, warp_grid).fit(h=0.15, reparam_grid=grid)

In [8]:
Z_LP, Q_LP, X_LP = ConstrainedLocalPolynomialRegression(Y, warp_grid, adaptative=False).fit(0.05, reparam_grid=None)
Z_LP_reparam, Q_LP_reparam, X_LP_reparam = ConstrainedLocalPolynomialRegression(Y, warp_grid, adaptative=False).fit(0.15, reparam_grid=grid)

In [9]:
visu.plot_3D([Q[:,:,0], Q_GS[:,:,0], Q_GS_reparam[:,:,0], Q_LP_reparam[:,:,0], Q_LP[:,:,0]], ['mu_T', 'T_GS', 'T_GS_rep', 'T_LP_rep', 'T_LP'])
visu.plot_3D([Q[:,:,1], Q_GS[:,:,1], Q_GS_reparam[:,:,1], Q_LP_reparam[:,:,1], Q_LP[:,:,1]], ['mu_N', 'N_GS', 'N_GS_rep', 'N_LP_rep', 'N_LP'])
visu.plot_3D([Q[:,:,2], Q_GS[:,:,2], Q_GS_reparam[:,:,2], Q_LP_reparam[:,:,2], Q_LP[:,:,2]], ['mu_B', 'B_GS', 'B_GS_rep', 'B_LP_rep', 'B_LP'])


In [10]:
h_grid = np.array([0.015, 0.02, 0.1, 0.15, 1])
h_opt, err_h = GramSchmidtOrthogonalization(Y, warp_grid).grid_search_CV_optimization_bandwidth(bandwidth_grid=h_grid, K_split=20)
print(err_h)

Optimal smoothing parameter h find by cross-validation: 0.15
[1.18203533e-04 6.91398368e-05 3.69674463e-05 3.29546341e-05
 1.50569093e-04]


In [11]:
h_grid = np.array([0.05, 0.1, 0.15, 0.2, 0.3, 1])
h_opt, err_h = ConstrainedLocalPolynomialRegression(Y, warp_grid, adaptative=False).grid_search_CV_optimization_bandwidth(bandwidth_grid=h_grid, K_split=5)
print(err_h)

Optimal smoothing parameter h find by cross-validation: 0.15
[0.00018632 0.0002843  0.00013023 0.00013309 0.00023775 0.02521713]
